##### Импорты

In [16]:
import random
import math

### Шифры гаммирования

#### Модуль 2

In [17]:
def gammaCipher(text, key):
    result = ''
    for i in range(len(text)):
        result += chr(ord(text[i]) ^ ord(key[i]))
    return result

text = 'Isyuk'
key = 'mehhh'
encrypt = gammaCipher(text, key)
decrypt = gammaCipher(encrypt, key)
print(f'''Ключ:                {key}\t{''.join([(10 - len(bin(ord(key[i])))) * ' ' + bin(ord(key[i]))[2:] for i in range(len(key))])}
Исходный текст:      {text}\t{''.join([(10 - len(bin(ord(text[i])))) * ' ' + bin(ord(text[i]))[2:] for i in range(len(text))])}
Зашифрованный текст: {encrypt}\t{''.join([(10 - len(bin(ord(encrypt[i])))) * ' ' + bin(ord(encrypt[i]))[2:] for i in range(len(encrypt))])}
Дешифрованный текст: {decrypt}\t{''.join([(10 - len(bin(ord(decrypt[i])))) * ' ' + bin(ord(decrypt[i]))[2:] for i in range(len(decrypt))])}
''')

Ключ:                mehhh	 1101101 1100101 1101000 1101000 1101000
Исходный текст:      Isyuk	 1001001 1110011 1111001 1110101 1101011
Зашифрованный текст: $	  100100   10110   10001   11101      11
Дешифрованный текст: Isyuk	 1001001 1110011 1111001 1110101 1101011



#### Модуль N

In [18]:

def gammaCipherEncrypt(text, key):
    result = ''
    printData = 'Шифрование:'
    for i in range(len(text)):
        piIslower = text[i].islower()
        pi = ord(text[i]) - (ord('a') if piIslower else ord('A'))
        ki = ord(key[i]) - ord('a')
        char = chr(((pi + ki) % 26) + (ord('a') if piIslower else ord('A')))
        result += char
        printData += f'\n({pi:2d} + {ki:2d}) mod 26 = {(pi + ki) % 26:2d} [{char}]'
    return result, printData

def gammaCipherDecrypt(text, key):
    result = ''
    printData = 'Расшифровка:'
    for i in range(len(text)):
        ciIslower = text[i].islower()
        ci = ord(text[i]) - (ord('a') if ciIslower else ord('A'))
        ki = ord(key[i]) - ord('a')
        char = chr(((ci + 26 - ki) % 26) + (ord('a') if ciIslower else ord('A')))
        result += char
        printData += f'\n({ci:2d} + 26 - {ki:2d}) mod 26 = {(ci + 26 - ki) % 26:2d} [{char}]'
    return result, printData

text = 'Isyuk'
key = 'cipher'
encrypt, encryptionLog = gammaCipherEncrypt(text, key)
decrypt, decryptionLog = gammaCipherDecrypt(encrypt, key)
print(f'''Ключ:                {key}   
Исходный текст:      {text}
{encryptionLog}
Зашифрованный текст: {encrypt}
{decryptionLog}
Дешифрованный текст: {decrypt}
''')


Ключ:                cipher   
Исходный текст:      Isyuk
Шифрование:
( 8 +  2) mod 26 = 10 [K]
(18 +  8) mod 26 =  0 [a]
(24 + 15) mod 26 = 13 [n]
(20 +  7) mod 26 =  1 [b]
(10 +  4) mod 26 = 14 [o]
Зашифрованный текст: Kanbo
Расшифровка:
(10 + 26 -  2) mod 26 =  8 [I]
( 0 + 26 -  8) mod 26 = 18 [s]
(13 + 26 - 15) mod 26 = 24 [y]
( 1 + 26 -  7) mod 26 = 20 [u]
(14 + 26 -  4) mod 26 = 10 [k]
Дешифрованный текст: Isyuk



#### Регистр сдвига c линейной обратной связью и алгоритм Блюм-Блюм-Шуба

In [35]:
def lfsr(seed, taps):
    index = 0
    while True:
        feedback = sum([seed[tap] for tap in taps]) % 2
        yield seed
        # yield seed[-1]
        # print(f'{index:>2d}) {seed} g[{seed[-1]}] {[seed[tap] for tap in taps]} b7[{sum([seed[tap] for tap in taps]) % 2}]')
        index += 1
        seed = [feedback] + seed[:-1]

def blumBlumShub(n, seed, taps):
    l = lfsr(seed, taps)
    result = []
    for _ in range(n):
        result.append(next(l))
    return result

def getTaps(polynomial):
    return [i for i, x in enumerate(polynomial) if x == '1']

def encrypt_surname(text, n, polynomial, initialValue):
    encryptionData = 'Шифрование:\n'
    seed = [int(x) for x in bin(initialValue)[2:].zfill(8)]
    taps = getTaps(polynomial)
    gamma = blumBlumShub(n, seed, taps)
    encrypted = ''.join([chr(ord(c) ^ (gamma[i])[-1]) for i, c in enumerate(text)])
    encryptionData += f'Таблица генерации гамм:  ({n} - итераций)\n'
    encryptionData += f' № |  Регистр  |      XOR(b7)      | Гамма | Символы | Результат\n'
    encryptionData += '-' * 65
    encryptedStr = ''
    for i, bit in enumerate(gamma):
        seedStr = ''.join(str(item) for item in gamma[i])
        gammaStr = str(bit[-1])
        char = text[i % len(text)]
        encryptedChar = chr(ord(char) ^ bit[-1])
        encryptedStr += encryptedChar
        xor = ' ⊻ '.join([seedStr[i] for i in taps])
        xorRes = sum(int(seedStr[i]) for i in taps) % 2
        encryptionData += f'\n{i + 1:^3}| {seedStr:^10}|{xor:^15}={xorRes:>2} | {gammaStr:^5} | {char:^3}>{encryptedChar:^3} | {encryptedStr}'
        seed = [bit] + seed[:-1]
    return encrypted, encryptionData

def decrypt_surname(text, n, polynomial, initialValue):
    decryptionData = 'Расшифровка:\n'
    seed = [int(x) for x in bin(initialValue)[2:].zfill(8)]
    taps = getTaps(polynomial)
    gamma = blumBlumShub(n, seed, taps)
    decrypted = ''.join([chr(ord(c) ^ (gamma[i])[-1]) for i, c in enumerate(text)])
    decryptionData += f'Таблица генерации гамм:  ({n} - итераций)\n'
    decryptionData += f' № |  Регистр  |      XOR(b7)      | Гамма | Символы | Результат\n'
    decryptionData += '-' * 65
    decryptedStr = ''
    for i, bit in enumerate(gamma):
        seedStr = ''.join(str(item) for item in gamma[i])
        gammaStr = str(bit[-1])
        char = text[i % len(text)]
        decryptedChar = chr(ord(char) ^ bit[-1])
        decryptedStr += decryptedChar
        xor = ' ⊻ '.join([seedStr[i] for i in taps])
        xorRes = sum(int(seedStr[i]) for i in taps) % 2
        decryptionData += f'\n{i + 1:^3}| {seedStr:^10}|{xor:^15}={xorRes:>2} | {gammaStr:^5} | {char:^3}>{decryptedChar:^3} | {decryptedStr}'
        seed = [bit] + seed[:-1]
    return decrypted, decryptionData


text = 'Исюк'
n = 10
polynomial = '10001110'

srcValue = [c for c in text.encode('cp1251')]
initialValue = srcValue[0]
for val in srcValue[1:]:
    initialValue ^= val
initialValueCalc = 'Расчет исходного значения:\n'
for i in range(len(text)):
    initialValueCalc += f'{text[i]}\t{bin(srcValue[i])[2:]:>8}\n'
initialValueCalc += f'\t{bin(initialValue)[2:]:>8}\n'
initialValueCalc += f'Исходное значение: {bin(initialValue)[2:]} = {initialValue}\n'

encrypt, encryptionLog = encrypt_surname(text, n, polynomial, initialValue)
decrypt, decryptionLog = decrypt_surname(encrypt, n, polynomial, initialValue)
print(f'''Исходный текст:      {text}
{initialValueCalc}
{encryptionLog}
Зашифрованный текст: {encrypt}

{decryptionLog}
Дешифрованный текст: {decrypt}
''')


Исходный текст:      Исюк
Расчет исходного значения:
И	11001000
с	11110001
ю	11111110
к	11101010
	  101101
Исходное значение: 101101 = 45

Шифрование:
Таблица генерации гамм:  (10 - итераций)
 № |  Регистр  |      XOR(b7)      | Гамма | Символы | Результат
-----------------------------------------------------------------
 1 |  00101101 | 0 ⊻ 1 ⊻ 1 ⊻ 0 = 0 |   1   |  И > Й  | Й
 2 |  00010110 | 0 ⊻ 0 ⊻ 1 ⊻ 1 = 0 |   0   |  с > с  | Йс
 3 |  00001011 | 0 ⊻ 1 ⊻ 0 ⊻ 1 = 0 |   1   |  ю > я  | Йся
 4 |  00000101 | 0 ⊻ 0 ⊻ 1 ⊻ 0 = 1 |   1   |  к > л  | Йсял
 5 |  10000010 | 1 ⊻ 0 ⊻ 0 ⊻ 1 = 0 |   0   |  И > И  | ЙсялИ
 6 |  01000001 | 0 ⊻ 0 ⊻ 0 ⊻ 0 = 0 |   1   |  с > р  | ЙсялИр
 7 |  00100000 | 0 ⊻ 0 ⊻ 0 ⊻ 0 = 0 |   0   |  ю > ю  | ЙсялИрю
 8 |  00010000 | 0 ⊻ 0 ⊻ 0 ⊻ 0 = 0 |   0   |  к > к  | ЙсялИрюк
 9 |  00001000 | 0 ⊻ 1 ⊻ 0 ⊻ 0 = 1 |   0   |  И > И  | ЙсялИрюкИ
10 |  10000100 | 1 ⊻ 0 ⊻ 1 ⊻ 0 = 0 |   0   |  с > с  | ЙсялИрюкИс
Зашифрованный текст: Йсял

Расшифровка:
Таблица генерации гамм